## Project 2: Visual Search System
### 1 Image Feature Extraction
#### (a) Extract 2000 SIFT features from 149 database images with each of 50 different buildings appearing in 2,3 or 4 of them.

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import os
import pickle
from sklearn.cluster import KMeans

In [7]:
# Import image
img_dir = 'server'
img_list = os.listdir(img_dir)
img_list.sort(key=lambda x: int((x.split('_')[0])[3:]))

# To match number of object and index of list, des_all[0] is empty and
# descriptors of object i are in des_all[i]
des_all = [[] for i in range(51)]
nfeature = 2000
for i in range(len(img_list)):

    obj = int((img_list[i].split('_')[0])[3:])

    img_path = os.path.join(img_dir, img_list[i])
    img = cv.imread(img_path)

    # Extract 3000 feature vectors from each image
    sift = cv.xfeatures2d.SIFT_create(nfeatures=nfeature, contrastThreshold=0.05, edgeThreshold=10)
    _, des = sift.detectAndCompute(img, None)
    if des.shape[0] < nfeature:
        print("Warning")
    des_all[obj].append(des)

# Adjust des_all to a matrix where each row represents a feature descriptor, with the label of object attached to the last element of the descriptor
database = np.zeros((nfeature*149, 129))
base_label = np.zeros(nfeature*149)
k = 0
for i in range(1, 51):
    des_obj = des_all[i]
    for j in range(len(des_obj)):
        des_one = des_obj[j]
        for m in range(nfeature):
            database[k, :128] = des_one[m, :]
            database[k, -1] = i
            k += 1

# Save database data to file using pickle
file = open('database.txt', 'wb')
pickle.dump(database, file)
file.close()

#### (b) Extract 2000 SIFT features from 50 query images of 50 different buildings, and save the data to folder Query_des

In [3]:
### Extract and save features of clients
nfeature = 2000
sift = cv.xfeatures2d.SIFT_create(nfeatures=nfeature, contrastThreshold=0.05, edgeThreshold=10)
nof = np.zeros(50)
for s in range(1, 51):
    im_path = 'client/'+'obj'+str(s)+'_t1.JPG'
    im = cv.imread(im_path)
    _, des = sift.detectAndCompute(im, None)
    nof[s-1] = des.shape[0]
    with open('./Query_des/des_client{}.pkl'.format(s), 'wb') as f:
            pickle.dump(des, f, pickle.HIGHEST_PROTOCOL)

### 2 Vocabulary Tree Construction
#### Recursively build the vocabulary tree which separate the 2000*149 feature vectors into #b to the power of #depth number of clusters a.k.a. visual vocabularies

In [ ]:
def hi_kmeans(data, b, depth, icenter, ileaf, center):
    '''
    :param data: The SIFT features from the database objects
    :param b: The branch number of vocabulary tree
    :param depth: The number of levels of the vocabulary tree
    :param icenter: The center of all clusters that is divided in every level
    :param ileaf: The leaf nodes a.k.a. visual vocabularies of the tree
    :param center: A temporary variable to store the centers of the clusters grown by a father cluster
    :return: icenter, ileaf
    '''
    if depth > 0:
        if data.shape[0] < b:
            icenter.append(center)
            for cnt in range(b):
                hi_kmeans(data, b, depth - 1, icenter, ileaf, center)
        else:
            kmeans = KMeans(n_clusters=b, random_state=0).fit(data[:, :128])
            label = kmeans.labels_
            center = list(kmeans.cluster_centers_)
            icenter.append(center)
            for clus_idx in range(b):
                data_in_clus = [m for m, n in enumerate(label) if n == clus_idx]
                hi_kmeans(data[data_in_clus], b, depth - 1, icenter, ileaf, center)
    else:
        ileaf.append(data)

    return icenter, ileaf

#### (a) Build the vocabulary tree with b=4, depth=3 and save the results to Hie_Tree_b4d3.txt

In [ ]:
b = 4
depth = 3
icenter, ileaf = hi_kmeans(database, b, depth, [], [], [])

file = open('Hie_Tree_b4d3.txt', 'wb')
pickle.dump((icenter, ileaf), file)
file.close()

#### (b) Build the vocabulary tree with b=4, depth=5 and save the results to Hie_Tree_b4d5.txt

In [ ]:
b = 4
depth = 5
icenter, ileaf = hi_kmeans(database, b, depth, [], [], [])

file = open('Hie_Tree_b4d5.txt', 'wb')
pickle.dump((icenter, ileaf), file)
file.close()

#### (c) Build the vocabulary tree with b=5, depth=7 and save the results to Hie_Tree_b5d7.txt

In [ ]:
b = 5
depth = 7
icenter, ileaf = hi_kmeans(database, b, depth, [], [], [])

file = open('Hie_Tree_b5d7.txt', 'wb')
pickle.dump((icenter, ileaf), file)
file.close()

## 3 Querying
#### Test the three kinds of vocabulary tree with query image and record the top-1 and top-5 recall rates over 50 objects

#### Define the required function for query and the computation of tf-idf/q vector/d vector

In [2]:
def query(b, depth, icenter, data, index, out, positionx):
    '''
    :param b: The branch number of vocabulary tree
    :param depth: The number of levels of the vocabulary tree
    :param icenter: The center of all clusters that is divided in every level
    :param data: The SIFT features from the query image
    :param index: The relative position of the descriptor in the tree in current level
    :param out: The label of the cluster a.k.a. visual words the descriptor finally belongs to
    :param positionx: The relative position of the descriptor in the tree in all upper levels
    :return: out
    '''
    if depth > 1:
        minimum = np.linalg.norm(data - icenter[index][0])
        position = 0
        for x in range(1, b):
            dist = np.linalg.norm(data - icenter[index][x])
            if dist < minimum:
                minimum = dist
                position = x
        index = int(index + 1 + position * ((np.power(b, depth-1) - 1)/(b-1)))
        positionx.append(position)
        query(b, depth-1, icenter, data, index, out, positionx)
    else:
        minimum = np.linalg.norm(data - icenter[index][0])
        position = 0
        for x in range(1, b):
            dist = np.linalg.norm(data - icenter[index][x])
            if dist < minimum:
                minimum = dist
                position = x
        positionx.append(position)
        loc = positionx[0] + 1
        for i in positionx[1:len(positionx)]:
            loc = loc * b - (b - (i + 1))
        out.append(loc-1)

    return out


def Compute_idf_d(words, n_o_w):
    '''
    :param words: All visual words contained in the database
    :param n_o_w: The number of all visual words in the database
    :return: idf, d
    '''
    # Calculate idf values of each Visual Word and d vector for the server database
    tf = np.zeros((n_o_w, 50))
    idf = np.zeros(n_o_w)
    d = np.zeros((n_o_w, 50))

    # idf values only depends on the database and the structure of the vocabulary tree
    for i in range(n_o_w):
        idf[i] = np.log2(50 / len(set(list(words[i][:, -1]))))
        for ob in words[i][:, -1]:
            ob = int(ob - 1)
            tf[i, ob] = tf[i, ob] + 1

    # Compute d vector (tf-idf weight) of each object and save it to d matrix as a column
    for j in range(50):
        tf[:, j] = tf[:, j] / np.sum(tf[:, j])
        d[:, j] = tf[:, j] * idf

    return idf, d


def Compute_q(word_q, n_o_w, idf):
    '''
    :param word_q: All visual words contained in a query image
    :param n_o_w: The number of all visual words in the database
    :param idf: The idf values given the database
    :return: q vector of the query image
    '''
    # Calculate q vector for each query image document
    tf = np.zeros(n_o_w)
    q = np.zeros(n_o_w)

    for i in range(len(word_q)):
        tf[word_q[i]] += 1
    tf /= np.sum(tf)

    for j in range(n_o_w):
        q[j] = tf[j] * idf[j]
    q.reshape(-1, 1)

    return q

#### Load the structure and node information of three trees

In [3]:
icenter_3 = [None] * 3
ileaf_3 = [None] * 3
with open('Hie_Tree_b4d3.txt', 'rb') as file:
    icenter_3[0], ileaf_3[0] = pickle.load(file)
with open('Hie_Tree_b4d5.txt', 'rb') as file:
    icenter_3[1], ileaf_3[1] = pickle.load(file)
with open('Hie_Tree_b5d7.txt', 'rb') as file:
    icenter_3[2], ileaf_3[2] = pickle.load(file)

#### Test the three kinds of vocabulary tree with query image and record the top-1 and top-5 recall rates over 50 objects

In [ ]:
rec_rate1 = np.zeros(3)
rec_rate5 = np.zeros(3)
b_all = [4, 4, 5]
depth_all = [3, 5, 7]
n_o_w_all = [np.power(4, 3), np.power(4, 5), np.power(5, 7)]

for i_tree in range(3):
    # Load the correct type of tree
    icenter = icenter_3[i_tree]
    ileaf = ileaf_3[i_tree]
    n_o_w = n_o_w_all[i_tree]
    b = b_all[i_tree]
    depth = depth_all[i_tree]

    # Initialization
    rec1 = np.zeros(50)
    rec5 = np.zeros(50)

    # Compute the idf and d vector of the database
    words = ileaf
    idf, d = Compute_idf_d(words, n_o_w)

    # Testing each query image
    for j in range(50):
        #Load descriptors in each image for query
        des_q = pickle.load(open('./Query_des/des_client{}.pkl'.format(j+1), 'rb'))

        # Query the vocabulary tree to find the word cluster the descriptor belonging to
        word_q = []
        for feature in des_q:
            word = query(b, depth, icenter, feature, 0, [], [])
            word_q.append(word)

        # Compute the q vector for each query image
        q = Compute_q(word_q, n_o_w, idf)

        # Calculate the qd score between all possible match and select the smallest one as best match
        score = np.zeros(50)
        for k in range(50):
            d_doc = d[:, k]
            score[k] = np.linalg.norm((d_doc/np.linalg.norm(d_doc, ord=1) - q/np.linalg.norm(q, ord=1)), ord=1)
        top = np.argsort(score)

        # Count the number of correct matches when only top1 is the correct object
        if top[0] == j:
            rec1[j] = rec1[j] + 1

        # Count the number of correct matches as long as one in top5 is the correct object
        for m in range(5):
            if top[m] == j:
                rec5[j] = rec5[j] + 1

    # TOP 1 Accuracy
    rec_rate1[i_tree] = np.mean(rec1)
    # TOP 5 Accuracy
    rec_rate5[i_tree] = np.mean(rec5)

In [5]:
print('The top1 recall rates when vocabulary tree with settings b4d3, b4d5 and b5d7 is:')
print(rec_rate1)
print('The top5 recall rates when vocabulary tree with settings b4d3, b4d5 and b5d7 is:')
print(rec_rate5)

The top1 recall rates when vocabulary tree with settings b4d3, b4d5 and b5d7 is:
[0.1  0.74 0.84]
The top5 recall rates when vocabulary tree with settings b4d3, b4d5 and b5d7 is:
[0.36 0.8  0.9 ]


#### Test the b=5, depth=7 vocabulary tree with 50, 70 or 90 percent of query features and record the top-1 and top-5 recall rates over 50 objects

In [11]:
# Load the information of correct type of tree
with open('Hie_Tree_b5d7.txt', 'rb') as file:
    icenter, ileaf = pickle.load(file)

# Define query factors
query_factor = [0.5, 0.7, 0.9]

# Compute the idf and d vector of the database
b = 5
depth = 7
words = ileaf
n_o_w = np.power(b, depth)  # number of visual words
idf, d = Compute_idf_d(words, n_o_w)

# Querying
rec_b5d7_top1 = np.zeros(len(query_factor))
rec_b5d7_top5 = np.zeros(len(query_factor))

for r in range(len(query_factor)):
    q_factor = query_factor[r]

    # Initialization
    rec1 = np.zeros(50)
    rec5 = np.zeros(50)

    # Randomly select query_factor amount of descriptors in each image for query
    for j in range(50):
        des_q = pickle.load(open('./Query_des/des_client{}.pkl'.format(j+1), 'rb'))
        ran_idx = np.random.randint(des_q.shape[0], size=round(q_factor*des_q.shape[0]))
        des_q = des_q[ran_idx]

        # Query the vocabulary tree to find the word cluster the descriptor belonging to
        word_q = []
        for feature in des_q:
            word = query(b, depth, icenter, feature, 0, [], [])
            word_q.append(word)

        # Compute the q vector for each query image
        q = Compute_q(word_q, n_o_w, idf)

        # Calculate the qd score between all possible match and select the smallest one as best match
        score = np.zeros(50)
        for k in range(50):
            d_doc = d[:, k]
            score[k] = np.linalg.norm((d_doc/np.linalg.norm(d_doc, ord=1) - q/np.linalg.norm(q, ord=1)), ord=1)
        top = np.argsort(score)

        # Count the number of correct matches when only top1 is the correct object
        if top[0] == j:
            rec1[j] = rec1[j] + 1

        # Count the number of correct matches as long as one in top5 is the correct object
        for m in range(5):
            if top[m] == j:
                rec5[j] = rec5[j] + 1

    # TOP 1 Accuracy
    rec_b5d7_top1[r] = np.mean(rec1)
    # TOP 5 Accuracy
    rec_b5d7_top5[r] = np.mean(rec5)

In [12]:
print('The top1 recall rates when query rate is 0.5, 0.7, 0.9 for vocabulary tree with settings b5d7 is:')
print(rec_b5d7_top1)
print('The top5 recall rates when query rate is 0.5, 0.7, 0.9 for vocabulary tree with settings b5d7 is:')
print(rec_b5d7_top5)

The top1 recall rates when query rate is 0.5, 0.7, 0.9 for vocabulary tree with settings b5d7 is:
[0.8  0.84 0.84]
The top5 recall rates when query rate is 0.5, 0.7, 0.9 for vocabulary tree with settings b5d7 is:
[0.9  0.94 0.92]
